In [21]:
import cv2
import numpy as np

In [22]:
yoloCfg = "/home/rko8001/Downloads/BTP/btp-models/object-detection/yolov3-tiny.cfg"
yoloWeights = "/home/rko8001/Downloads/BTP/btp-models/object-detection/yolov3-tiny.weights"
coco_names = "/home/rko8001/Downloads/BTP/btp-models/object-detection/coco.names"
'''
    For Object detection, we have taken a threshold of 5 frames. 
    If the object is detected once, then next 'x' consecutive frames will not be predicted
    due to multiple predictions on same apple.
'''
framesToSkip = 5

''' 
    Width Threshold, if apple is present in between this threshold
    then it will be marked as detected
'''
widthBegin, widthEnd = 300, 320


'''
    While object detection, we have taken a confidence threshold of 0.4
    with this confidence, we are able to detect the object with good accuracy and speed
'''
confidenceInterval = 0.4

In [23]:
net = cv2.dnn.readNet(yoloWeights, yoloCfg)

classes = []
with open(coco_names, "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getUnconnectedOutLayersNames()

In [24]:
def extractDimensions(detection, frame):
    height, width, _ = frame.shape
    center_x = int(detection[0] * width)
    center_y = int(detection[1] * height)
    w = int(detection[2] * width)
    h = int(detection[3] * height)
    x = int(center_x - w / 2)
    y = int(center_y - h / 2)
    return x, y, w, h, center_x, center_y

In [25]:
def objectDetection(frame): 
    global confidenceInterval, widthBegin, widthEnd

    inMiddle = False
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(layer_names)

    # Process the outputs and draw bounding boxes
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence >= confidenceInterval:  # Adjust the confidence threshold as needed
                x, y, w, h, center_x, center_y = extractDimensions(detection, frame)
                cv2.rectangle(frame, (x, y), (x + w, y+h), (0, 255, 0), 2)

                if (center_x >= widthBegin and center_x <= widthEnd): 
                    inMiddle = True
                break

    return frame, inMiddle

In [33]:
def display_camera(camera):
    global framesToSkip
    sortCategory, prediction, previousFrame, skipFrame = "color", None, False, 0

    while True:
        success, frame = camera.read()
        previousFrame = frame
        if not success:
            break

        newFrame, objectDetected = objectDetection(frame)

        if objectDetected == True and skipFrame == 0: 
            # prediction = predictor(frame, sortCategory)
            print("aa")
        elif objectDetected == False:
            skipFrame = 0
        else:
            skipFrame = (skipFrame + 1)% framesToSkip # no of frames to skip


    camera.release()
    cv2.destroyAllWindows()


In [35]:
def checkFrameSkipLogic(framesOutput):
    global framesToSkip
    previousFrame, prediction, skipFrame = False, None, 0
    for i in range(len(framesOutput)):
        objectDetected = framesOutput[i]
        print(objectDetected)

        if objectDetected == True and skipFrame == 0: 
            print("Prediction on frame: ", i)
        elif objectDetected == False:
            skipFrame = 0
        else:
            skipFrame = (skipFrame + 1)% framesToSkip # no of frames to skip
        

In [38]:
def logicCheck():
    framesOutput = [True, False, True, False, False, False, False, False, False, False]
    checkFrameSkipLogic(framesOutput)

In [29]:
def main():
    camera = cv2.VideoCapture(2)
    display_camera(camera)

In [39]:
if __name__ == "__main__":
    # main()
    logicCheck()

True
Prediction on frame:  0
False
True
Prediction on frame:  2
False
False
False
False
False
False
False
